# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846800451228                   -0.70    4.5         
  2   -7.852311551617       -2.26       -1.53    1.0   24.9ms
  3   -7.852614380603       -3.52       -2.56    1.5   86.7ms
  4   -7.852645937001       -4.50       -2.87    2.8   34.0ms
  5   -7.852646468710       -6.27       -3.13    1.2   25.8ms
  6   -7.852646679725       -6.68       -4.17    1.0   24.7ms
  7   -7.852646686695       -8.16       -5.12    2.2   33.1ms
  8   -7.852646686726      -10.50       -5.53    1.5   28.8ms
  9   -7.852646686726   +  -12.22       -5.44    1.2   27.1ms
 10   -7.852646686730      -11.40       -6.74    1.0   25.7ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846802155537                   -0.70           4.5         
  2   -7.852553311921       -2.24       -1.63   0.80    2.2    280ms
  3   -7.852639246220       -4.07       -2.74   0.80    1.0   23.1ms
  4   -7.852646499172       -5.14       -3.37   0.80    2.0   28.7ms
  5   -7.852646678452       -6.75       -4.27   0.80    1.8   27.4ms
  6   -7.852646686605       -8.09       -4.79   0.80    2.2   30.8ms
  7   -7.852646686720       -9.94       -5.59   0.80    1.2   25.0ms
  8   -7.852646686730      -11.03       -6.29   0.80    2.0   29.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.398744e+01     3.401080e+00
 * time: 0.40676212310791016
     1     1.607735e+00     1.487794e+00
 * time: 0.6094880104064941
     2    -1.464614e+00     1.795400e+00
 * time: 0.6332619190216064
     3    -3.818594e+00     1.657837e+00
 * time: 0.6671810150146484
     4    -5.239990e+00     1.364904e+00
 * time: 0.7010130882263184
     5    -6.820457e+00     9.995439e-01
 * time: 0.7349460124969482
     6    -7.434940e+00     3.538301e-01
 * time: 0.7687890529632568
     7    -7.648126e+00     1.275642e-01
 * time: 0.7923500537872314
     8    -7.738218e+00     1.570176e-01
 * time: 0.8159260749816895
     9    -7.785697e+00     8.707863e-02
 * time: 0.839616060256958
    10    -7.817463e+00     7.627000e-02
 * time: 0.8635361194610596
    11    -7.834654e+00     6.543163e-02
 * time: 0.8874509334564209
    12    -7.844786e+00     4.499375e-02
 * time: 0.9115350246429443
    13    -7.849162e+00     4.490726e-02
 * time: 0.9354639053

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846858817110                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645915143                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.70s
  3   -7.852646686730      -13.34       -7.25    171ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 3.455585237044646e-7
|ρ_newton - ρ_scfv| = 8.158059527326041e-8
|ρ_newton - ρ_dm|   = 6.269029354273328e-10
